In [ ]:
# Install required libraries
!pip install tensorflow keras opencv-python scikit-image matplotlib


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, BatchNormalization, ReLU, Add, Input
from tensorflow.keras.optimizers import Adam
import cv2
import numpy as np
import matplotlib.pyplot as plt
from skimage import exposure
from google.colab import files
import os


In [ ]:
# Upload moonimage dataset from your local machine
uploaded = files.upload()

# Assuming the uploaded files are in a zip format, extract them
import zipfile
for filename in uploaded.keys():
    if filename.endswith('.zip'):
        with zipfile.ZipFile(filename, 'r') as zip_ref:
            zip_ref.extractall('./moonimage')

# List files to ensure they are uploaded correctly
import os
data_path = './moonimage/'
print(os.listdir(data_path))


In [ ]:
def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder,filename))
        if img is not None:
            # Convert the image to grayscale and normalize
            img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            img = img.astype('float32') / 255.0
            images.append(img)
    return np.array(images)

# Load the images
moon_images = load_images_from_folder(data_path)

# Reshape for the neural network (assuming images are 256x256 pixels)
moon_images = np.reshape(moon_images, (-1, 256, 256, 1))

# Split into training and test sets
from sklearn.model_selection import train_test_split
train_images, test_images = train_test_split(moon_images, test_size=0.2, random_state=42)


In [ ]:
def build_model(input_shape):
    model = Sequential()

    # Convolutional Layers
    model.add(Input(shape=input_shape))
    model.add(Conv2D(64, kernel_size=(3, 3), padding='same'))
    model.add(ReLU())

    # Residual Blocks
    for _ in range(5):
        skip = model.layers[-1].output
        x = Conv2D(64, kernel_size=(3, 3), padding='same')(skip)
        x = ReLU()(x)
        x = Conv2D(64, kernel_size=(3, 3), padding='same')(x)
        x = Add()([x, skip])
        model.add(BatchNormalization()(x))

    # Final Conv Layer
    model.add(Conv2D(1, kernel_size=(3, 3), padding='same'))

    return model

model = build_model(input_shape=(256, 256, 1))
model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error', metrics=['accuracy'])
model.summary()


In [ ]:
# Train the model
history = model.fit(train_images, train_images,
                    epochs=50,
                    batch_size=16,
                    validation_data=(test_images, test_images))

# Plot training history
plt.plot(history.history['loss'], label='Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()


In [ ]:
# Test the model
test_output = model.predict(test_images)

# Display a few sample results
n = 5  # Number of samples to display
plt.figure(figsize=(15, 10))
for i in range(n):
    plt.subplot(2, n, i + 1)
    plt.imshow(test_images[i].reshape(256, 256), cmap='gray')
    plt.title("Original")
    plt.axis('off')

    plt.subplot(2, n, i + n + 1)
    plt.imshow(test_output[i].reshape(256, 256), cmap='gray')
    plt.title("Enhanced")
    plt.axis('off')
plt.show()


In [ ]:
# Save the trained model
model.save('low_light_image_enhancement.h5')
